In [341]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import math
import sklearn
from plotly import graph_objects as go
from plotly import express as px
import seaborn as sns
from jupyterthemes import jtplot
jtplot.style()

datapath = ""
books = pd.read_csv(datapath + "books.csv")
toRead = pd.read_csv(datapath + "to_read.csv")
tags = pd.read_csv(datapath + "tags.csv")
bookTags = pd.read_csv(datapath + "book_tags.csv")
ratings = pd.read_csv(datapath + "ratings.csv")

print(len(bookTags))

print(bookTags.columns)
bookTags.head()

999912
Index(['goodreads_book_id', 'tag_id', 'count'], dtype='object')


,goodreads_book_id,tag_id,count
0,1,30574,167697
1,1,11305,37174
2,1,11557,34173
3,1,8717,12986
4,1,33114,12716


# Tag based recommendation

In [189]:
booksTagLists = {}
for i in bookTags.values:
    if i[0] in booksTagLists: booksTagLists[i[0]].append([i[1], i[2]])
    else:                     booksTagLists[i[0]] = [[i[1], i[2]]]

In [257]:
from sklearn import preprocessing
def Tags2Vec(tagList):
    ret = np.zeros(len(tags))
    for i in tagList:
        ret[i[0]] = 1
    return ret

def Tags_similarity(tagList1, tagList2):
    return cosine_similarity([Tags2Vec(tagList1)], [Tags2Vec(tagList2)])[0][0]

In [452]:
from sklearn.metrics.pairwise import cosine_similarity
selected = 485894
rec = []
for b in booksTagLists:
    sim = Tags_similarity(booksTagLists[selected], booksTagLists[b])
    if sim > 0.65:
        rec.append([b, sim])

In [453]:
rec = np.array(rec)
ind=np.argsort(rec[:,1])
rec2 = np.array(rec)[ind][::-1]
print("Based on ", books[books["book_id"] ==selected]["original_title"].values[0], " we recommend : ")
for i in rec2:
    print(books[books["book_id"] ==i[0]]["original_title"].values[0], " : ", i[1])

Based on  Die Verwandlung  we recommend : 
Die Verwandlung  :  1.0000000000000007
Der Prozeß  :  0.7400000000000005
Heart of Darkness  :  0.6700000000000005


# Author based recommendation

In [291]:
AuthorList = {}
for i in books["authors"]:
    authors = i.split(", ")
    for a in authors:
        AuthorList[a] = len(AuthorList)

In [292]:
def Auth2Vec(authors):
    ret = np.zeros(len(AuthorList))
    for i in authors:
        ret[AuthorList[i]] = 1
    return ret

def Authors_similarity(authList1, authList2):
    return cosine_similarity([Auth2Vec(authList1)], [Auth2Vec(authList2)])[0][0]

In [424]:
from sklearn.metrics.pairwise import cosine_similarity
selected = 5954
rec = []
sb = books[books["book_id"] == selected].authors.values[0].split(", ")
for index, b in books.iterrows():
    i = b.authors.split(", ")
    sim = Authors_similarity(sb, i)
    if sim > 0.45:
        rec.append([b.original_title, sim])

Siddhartha
Der Steppenwolf: Erzählung
Demian: Die Geschichte einer Jugend
Narziß und Goldmund


In [429]:
rec = np.array(rec)
ind=np.argsort(rec[:,1])
rec2 = np.array(rec)[ind][::-1]
print("Based on ", books[books["book_id"] ==selected]["original_title"].values[0], " we recommend : ")
for i in rec2:
    print(i[0], " : ", i[1])

Based on  Narziß und Goldmund  we recommend : 
Narziß und Goldmund  :  0.9999999999999998
Demian: Die Geschichte einer Jugend  :  0.7071067811865475
Der Steppenwolf: Erzählung  :  0.4999999999999999
Siddhartha  :  0.4999999999999999


In [451]:
from sklearn.metrics.pairwise import cosine_similarity
selected = 1
rec = []
sb = ["Franz Kafka"]
for index, b in books.iterrows():
    i = b.authors.split(", ")
    sim = Authors_similarity(sb, i)
    
    if sim > 0.65:
        print(b)
        rec.append([b.original_title, sim])

rec = np.array(rec)
ind=np.argsort(rec[:,1])
rec2 = np.array(rec)[ind][::-1]
print("Based on ", books[books["book_id"] ==selected]["original_title"].values[0], " we recommend : ")
for i in rec2:
    print(i[0], " : ", i[1])

id                                                                         213
book_id                                                                 485894
best_book_id                                                            485894
work_id                                                                2373750
books_count                                                               1316
isbn                                                                 553213695
isbn13                                                             9.78055e+12
authors                                          Franz Kafka, Stanley Corngold
original_publication_year                                                 1915
original_title                                                 Die Verwandlung
title                                                        The Metamorphosis
language_code                                                              eng
average_rating                                      

id                                                                        5838
book_id                                                                2102600
best_book_id                                                           2102600
work_id                                                               18280032
books_count                                                                 26
isbn                                                                 765316986
isbn13                                                             9.78077e+12
authors                                                            John Scalzi
original_publication_year                                                 2008
original_title                                                      Zoe's Tale
title                                           Zoe's Tale (Old Man's War, #4)
language_code                                                              eng
average_rating                                      

id                                                                        8294
book_id                                                                   7081
best_book_id                                                              7081
work_id                                                               18280036
books_count                                                                 14
isbn                                                                 765309416
isbn13                                                             9.78077e+12
authors                                                            John Scalzi
original_publication_year                                                 2006
original_title                                             The Android's Dream
title                                                      The Android's Dream
language_code                                                            en-US
average_rating                                      

# Ratings based recommendation

In [376]:
RatingMatrix = {}
for index, r in ratings.iterrows():
    if r.book_id not in RatingMatrix: RatingMatrix[r.book_id] = [[r.user_id, r.rating]]
    else: RatingMatrix[r.book_id].append([r.user_id, r.rating])
    

In [389]:
def Rating2Vec(UserRatings):
    ret = np.zeros(len(ratings.user_id.values))
    for i in UserRatings:
        ret[i[0]] = i[1]
    return ret

def Ratings_similarity(ratings1, ratings2):
    return cosine_similarity([Rating2Vec(ratings1)], [Rating2Vec(ratings2)])[0][0]

In [396]:
Ratings_similarity(RatingMatrix[1], RatingMatrix[514])

0.06934222840689643

# Mixed recommendation

In [459]:
def Recommend_with_book(bookId, n=10):
    rec = []
    st = booksTagLists[bookId]
    wb = [0.8, 0.05, 0.15]
    if bookId in RatingMatrix: sr = RatingMatrix[bookId]
    else : wb = [0.85, 0, 0.15]
    sa = books[books["book_id"] == bookId].authors.values[0].split(", ")
    for index, b in books.iterrows():
        w = wb
        t = booksTagLists[b.book_id]
        a = b.authors.split(", ")
        
        if b.book_id not in RatingMatrix: w = [0.85, 0, 0.15]
        
        simT = Tags_similarity(st, t)
        if w[1] != 0: simR = Ratings_similarity(sr, RatingMatrix[b.book_id])
        else : simR = 0
        simA = Authors_similarity(sa, a)
        
        sim = (w[0] * simT + w[1] * simR + w[2] * simA)
        if sim > 0.5:
            rec.append([b.original_title, sim])
            
    rec = np.array(rec)
    ind=np.argsort(rec[:,1])
    srec = np.array(rec)[ind][::-1]
    return srec[:10]

In [460]:
bookId = 485894
rec = Recommend_with_book(bookId)
print("Based on ", books[books["book_id"] ==bookId]["original_title"].values[0], " we recommend : ")
for i in rec:
    print(i[0], " : ", i[1])

Based on  Die Verwandlung  we recommend : 
Die Verwandlung  :  1.0000000000000004
Der Prozeß  :  0.6902372435695799
In der Strafkolonie  :  0.6415660171779826
Die Verwandlung und andere Erzählungen  :  0.5882372435695798
Heart of Darkness  :  0.5695000000000005
Animal Farm: A Fairy Story  :  0.5440000000000004
Amerika  :  0.5425000000000003
The Old Man and the Sea  :  0.5355000000000004
Candide  :  0.5355000000000004
Im Westen nichts Neues  :  0.5355000000000004
